# OMERO Micro-SAM Workflow




## 1. Setup and Installation

In [1]:
# Import the streamlined package
import omero_annotate_ai
from omero_annotate_ai import (
    create_omero_connection_widget,
    create_workflow_widget,
    create_pipeline
)

# System imports
import os
from pathlib import Path

print(f"📦 omero-annotate-ai version: {omero_annotate_ai.__version__}")
# Check dependencies
try:
    import ezomero
    print(f"🔗 OMERO functionality: ✅ Available")
except ImportError:
    print(f"🔗 OMERO functionality: ❌ Install with: pip install -e .[omero]")

try:
    import keyring
    print(f"🔑 Keyring support: ✅ Available")
except ImportError:
    print(f"🔑 Keyring support: ⚠️ Not available (manual password entry only)")

📦 omero-annotate-ai version: 0.1.0
🔗 OMERO functionality: ✅ Available
🔑 Keyring support: ✅ Available


## 2. OMERO Connection

In [2]:
# Create and display the OMERO connection widget
print("🔌 OMERO Connection Setup")
print("Use the widget below to connect to your OMERO server:")
print("  • Fill in server details (host, username, password)")
print("  • Test connection before proceeding")
print("  • Choose password storage duration if desired")
print("  • Click 'Save & Connect' to establish connection")
print()

conn_widget = create_omero_connection_widget()
conn_widget.display()

print("\n📝 Next Step: Run the cell below after connecting to OMERO")

🔌 OMERO Connection Setup
Use the widget below to connect to your OMERO server:
  • Fill in server details (host, username, password)
  • Test connection before proceeding
  • Choose password storage duration if desired
  • Click 'Save & Connect' to establish connection

📄 Loaded configuration from connection history: root@localhost
🔐 Password loaded from keychain (no expiration)



📝 Next Step: Run the cell below after connecting to OMERO


In [3]:
# Get the OMERO connection
conn = conn_widget.get_connection()

if conn is None:
    raise ConnectionError("❌ No OMERO connection established. Please use the widget above to connect.")

print("✅ OMERO connection established!")
print(f"👤 User: {conn.getUser().getName()}")
print(f"🏢 Group: {conn.getGroupFromContext().getName()}")
print(f"🔐 Secure: {conn.isSecure()}")
print(f"🔗 Connection ready for workflow setup")

ConnectionError: ❌ No OMERO connection established. Please use the widget above to connect.

## 3. Workflow Configuration

In [ ]:
# Create and display the workflow widget
print("🔬 OMERO Annotation Workflow Setup")
print("Follow the sequential workflow below:")
print("  1. Select working directory")
print("  2. Choose OMERO container")
print("  3. Check existing annotation tables")
print("  4. Configure micro-SAM parameters")
print("  5. Save configuration")
print()

workflow_widget = create_workflow_widget(connection=conn)
workflow_widget.display()

print("\n📝 Next Step: Complete the workflow above, then run the cell below")

🔬 OMERO Annotation Workflow Setup
Follow the sequential workflow below:
  1. Select working directory
  2. Choose OMERO container
  3. Check existing annotation tables
  4. Configure micro-SAM parameters
  5. Save configuration




📝 Next Step: Complete the workflow above, then run the cell below


## 4. Pipeline Execution


In [8]:
# Get configuration from workflow widget
config = workflow_widget.get_config()

# Display configuration summary
print("\n📋 Configuration Summary:")
print(f"   📦 Container: {config.omero.container_type} (ID: {config.omero.container_id})")
print(f"   🎯 Training Set: {config.name}")
print(f"   🔬 Model: {config.ai_model.model_type}")
print(f"   📺 Channel: {config.spatial_coverage.channels}")
print(f"   📁 Output: {config.output.output_directory}")
print(f"   🔄 Resume from Table: {config.workflow.resume_from_table}")
print(f"   📖 Read-only Mode: {config.workflow.read_only_mode}")

if config.processing.use_patches:
    print(f"   🧩 Patches: {config.processing.patches_per_image} per image ({config.processing.patch_size[0]}×{config.processing.patch_size[1]})"
    )

if config.spatial_coverage.three_d:
    print(f"   🧊 3D processing: Enabled")


print(f"\n📊 Processing: {'All images' if config.training.segment_all else f'{config.training.train_n} training + {config.training.validate_n} validation'}")


📋 Configuration Summary:
   📦 Container: project (ID: 201)
   🎯 Training Set: micro_sam_foci_test_20250825_204521
   🔬 Model: vit_b_lm
   📺 Channel: [0]
   📁 Output: c:\Users\Maarten\Documents\Github\omero_annotate_ai\examples\omero_annotations
   🔄 Resume from Table: False
   📖 Read-only Mode: False
   🧩 Patches: 1 per image (512×512)

📊 Processing: 1 training + 1 validation


In [9]:
from omero_annotate_ai.core.annotation_config import load_config_from_yaml

#config = load_config_from_yaml(
#    "C:\\Users\\Maarten\\Documents\\Github\\omero_annotate_ai\\examples\\omero_annotations\\annotation_config.yaml"
#)


In [10]:
# Create pipeline and preview what will be processed
pipeline = create_pipeline(config, conn)

# Get container details
container_type = config.omero.container_type
container_id = config.omero.container_id

print(f"🔍 Validating {container_type} with ID {container_id}...")

# Validate container exists
container = conn.getObject(container_type.capitalize(), container_id)
if container is None:
    raise ValueError(f"{container_type.capitalize()} with ID {container_id} not found")

print(f"✅ Found {container_type}: {container.getName()}")
if container.getDescription():
    print(f"📝 Description: {container.getDescription()}")

🔍 Validating project with ID 201...
✅ Found project: foci_test


In [ ]:
# Run the complete annotation pipeline
print("🚀 Starting annotation pipeline...")
print(f"   🔬 Using micro-SAM model: {config.ai_model.model_type}")

if config.processing.batch_size == 0:
    print(f"   ⚡ Processing: All images in one batch")
else:
    print(f"   📦 Processing: Batches of {config.processing.batch_size} images")

print(f"   🎨 Napari will open for interactive annotation")
print(f"   📝 Close napari windows when annotation is complete")

try:
    # Run the complete workflow - this is the key call!
    table_id, processed_images = pipeline.run_full_workflow()
    
    print(f"\n🎉 Annotation pipeline completed successfully!")
    print(f"📋 Tracking table ID: {table_id}")
    
    if config.workflow.read_only_mode:
        print(f"💾 Annotations saved locally to: {config.output.output_directory}")
    else:
        print(f"☁️ Annotations uploaded to OMERO")
        
except Exception as e:
    print(f"Error during annotation pipeline: {e}")
    import traceback
    traceback.print_exc()
    raise

🚀 Starting annotation pipeline...
   🔬 Using micro-SAM model: vit_b_lm
   ⚡ Processing: All images in one batch
   🎨 Napari will open for interactive annotation
   📝 Close napari windows when annotation is complete
Loading images from project 201
Found 10 images
Processing 10 images with model: vit_b_lm
Preparing processing units...
Creating initial OMERO table...
📋 Created/replaced tracking table 'micro_sam_training_micro_sam_foci_test_20250825_204521' with 2 units
   Container: project 201
   Table ID: 1510
Created OMERO table with ID: 1510
object group 0
📊 Workflow status updated: 0/2 (0.0%) - pending
Found 2 unprocessed annotations

🔄 Processing batch 1/1
   📦 Batch size: 2
   📊 Config state before: 0/2 processed
   🎨 Running micro-SAM annotation...
📊 Loading 1 images using dask...
💾 Materializing dask arrays to numpy...
   Processing chunk 1/1


INFO:omero.gateway:Registered c82fc091-7876-4f4e-a1a4-f8c06b13f97e/e1d007f8-1565-4bab-b910-382039931de3omero.api.RawPixelsStore -t -e 1.1:tcp -h 172.19.0.8 -p 34317 -t 60000
INFO:omero.gateway:Unregistered c82fc091-7876-4f4e-a1a4-f8c06b13f97e/e1d007f8-1565-4bab-b910-382039931de3omero.api.RawPixelsStore -t -e 1.1:tcp -h 172.19.0.8 -p 34317 -t 60000
INFO:omero.gateway:Registered c82fc091-7876-4f4e-a1a4-f8c06b13f97e/c1ce2b31-90c1-4eab-a168-8edf82d11679omero.api.RawPixelsStore -t -e 1.1:tcp -h 172.19.0.8 -p 34317 -t 60000
INFO:omero.gateway:Unregistered c82fc091-7876-4f4e-a1a4-f8c06b13f97e/c1ce2b31-90c1-4eab-a168-8edf82d11679omero.api.RawPixelsStore -t -e 1.1:tcp -h 172.19.0.8 -p 34317 -t 60000


✅ Successfully loaded 1 images
Loaded 2D data shape: (1024, 1024)
📊 Loading 1 images using dask...
💾 Materializing dask arrays to numpy...
   Processing chunk 1/1
✅ Successfully loaded 1 images
Loaded 2D data shape: (1024, 1024)


Precompute state for files: 100%|██████████| 2/2 [00:15<00:00,  7.68s/it]


Precomputation took 16.681240558624268 seconds (= 00:17 minutes)
The first image to annotate is image number 0
Loading next image: at index 1
---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
File c:\Users\Maarten\miniforge3\envs\micro-sam\Lib\site-packages\psygnal\_signal.py:1196, in SignalInstance._run_emit_loop(self=<class 'psygnal._signal.SignalInstance'> instance, args=(False,))
   1195     with Signal._emitting(self):
-> 1196         self._run_emit_loop_inner()
        Exception trying to inspect frame. No more locals available.
   1197 except RecursionError as e:

File c:\Users\Maarten\miniforge3\envs\micro-sam\Lib\site-packages\psygnal\_signal.py:1225, in SignalInstance._run_emit_loop_immediate(self=<class 'psygnal._signal.SignalInstance'> instance)
   1224 for caller in self._slots:
-> 1225     caller.cb(args)
        args = (False,)
        caller = <StrongFunction on magicgu

INFO:omero.gateway:Registered c82fc091-7876-4f4e-a1a4-f8c06b13f97e/c153ea1c-1643-42e6-9f37-00943b2387a2omero.api.RawFileStore -t -e 1.1:tcp -h 172.19.0.8 -p 34317 -t 60000
INFO:omero.gateway:Unregistered c82fc091-7876-4f4e-a1a4-f8c06b13f97e/c153ea1c-1643-42e6-9f37-00943b2387a2omero.api.RawFileStore -t -e 1.1:tcp -h 172.19.0.8 -p 34317 -t 60000
INFO:omero.gateway:Registered c82fc091-7876-4f4e-a1a4-f8c06b13f97e/72a366ae-d87a-42b9-bf17-e530d6f8c35domero.api.RawFileStore -t -e 1.1:tcp -h 172.19.0.8 -p 34317 -t 60000
INFO:omero.gateway:Unregistered c82fc091-7876-4f4e-a1a4-f8c06b13f97e/72a366ae-d87a-42b9-bf17-e530d6f8c35domero.api.RawFileStore -t -e 1.1:tcp -h 172.19.0.8 -p 34317 -t 60000


✅ File annotation uploaded with ID: 1512
🔍 Step 4: Uploading ROI shapes
✅ Created 1 ROI shapes for image 454 with ID: 375
☁️ Uploaded annotations from c:\Users\Maarten\Documents\Github\omero_annotate_ai\examples\omero_annotations\annotations\seg_00000.tif to image 454
   Patch offset: (80, 75)
   File annotation ID: 1512
   ROI ID: 375
✅ Uploaded ROI for annotation_id '454_0_0_0' (label_id: 1512, roi_id: 375)
🔍 Step 1: Loading label image from c:\Users\Maarten\Documents\Github\omero_annotate_ai\examples\omero_annotations\annotations\seg_00001.tif
📋 Label image loaded: (512, 512), dtype: uint32
🏷️ Found 2 unique labels: [0 1]...
🔍 Step 2: Converting labels to ROI shapes...
✅ Created 1 ROI shapes from labels
🔍 Step 3: Uploading label file as attachment
✅ File annotation uploaded with ID: 1513
🔍 Step 4: Uploading ROI shapes
✅ Created 1 ROI shapes for image 460 with ID: 376
☁️ Uploaded annotations from c:\Users\Maarten\Documents\Github\omero_annotate_ai\examples\omero_annotations\annotatio

Traceback (most recent call last):
  File "C:\Users\Maarten\AppData\Local\Temp\ipykernel_35512\292021098.py", line 18, in <module>
    print(f"📊 Processed {len(processed_images)} images")
                          ^^^^^^^^^^^^^^^^^^^^^
TypeError: object of type 'AnnotationConfig' has no len()


TypeError: object of type 'AnnotationConfig' has no len()

In [ ]:
# Display results summary
print("📊 Pipeline Results Summary:")
print(f"   🎯 Training Set: {config.name}")
print(f"   📋 Tracking Table ID: {table_id}")
print(f"   📊 Images Processed: {len(processed_images)}")
print(f"   📦 Container: {config.omero.container_type} (ID: {config.omero.container_id})")
print(f"   🔬 Model Used: {config.ai_model.model_type}")
print(f"   📁 Output Location: {config.output.output_directory}")

# Show processed images
if processed_images:
    print(f"\n🖼️ Processed Images:")
    for i, img_obj in enumerate(processed_images[:10]):
        if img_obj:
            print(f"   {i + 1}. {img_obj.getName()} (ID: {img_obj.getId()})")

    if len(processed_images) > 10:
        print(f"   ... and {len(processed_images) - 10} more images")

print(f"\n✅ Pipeline completed successfully!")

In [ ]:
# Export configuration for future use
config_filename = f"annotation_config_{config.name}.yaml"
config_path = Path(config.output.output_directory) / config_filename

try:
    config.save_yaml(config_path)
    print(f"💾 Configuration saved to: {config_path}")
except Exception as e:
    print(f"⚠️ Could not save to output folder, saving to current directory")
    config.save_yaml(config_filename)
    print(f"💾 Configuration saved to: {config_filename}")

print(f"\n📋 Configuration Summary:")
print(f"   Name: {config.name}")
print(f"   Description: {config.description}")
print(f"   Output Directory: {config.output.output_directory}")
print(f"   Model Type: {config.ai_model.model_type}")

print(f"\n🔄 To reuse this configuration:")
print(f"```python")
print(f"from omero_annotate_ai import load_config")
print(f"config = load_config('{config_filename}')")
print(f"```")

In [ ]:
# Import training widget
from omero_annotate_ai import create_training_data_widget

print("Training Data Selection")
print("Select existing annotation tables for training:")
print()

# Create and display training data widget
training_widget = create_training_data_widget(connection=conn)
training_widget.display()

In [ ]:
# Get selected training table
selected_table_id = training_widget.get_selected_table_id()
selected_table_info = training_widget.get_selected_table_info()

if selected_table_id:
    print(f"Selected training table:")
    print(f"  Table ID: {selected_table_id}")
    print(f"  Table Name: {selected_table_info.get('name', 'Unknown')}")
    print(f"  Created: {selected_table_info.get('created', 'Unknown')}")
else:
    print("No training table selected.")

### Training Data Preparation (Future)

When the training data preparation function is implemented, you can use it like this:

```python
# Future implementation
from omero_annotate_ai import prepare_training_data_from_table

# Prepare training data automatically
training_info = prepare_training_data_from_table(
    conn=conn,
    table_id=selected_table_id,
    output_dir="./training_data"
)

# Use training_info for micro-SAM training
print(f"Training data ready at: {training_info['paths']}")
```

## 6. Cleanup

Close the OMERO connection when finished.

In [ ]:
# Close OMERO connection
if 'conn' in locals() and conn is not None:
    conn.close()
    print("🔌 OMERO connection closed")

print(f"\n🎉 Streamlined workflow completed!")
print(f"📊 Total images processed: {len(processed_images) if 'processed_images' in locals() else 0}")
print(f"💾 Configuration saved for future use")
print(f"✨ Ready for next annotation workflow!")